
**矩阵链和矩阵相容**  
给定一个矩阵链 $A_0, A_1, ..., A_l $,矩阵$A_i$的维度为$D_i * D_{i+1}$,此矩阵链中前后相接的矩阵相容(也就是前一个矩阵的列数等于后一个矩阵的行数,可以进行矩阵乘法).  

**矩阵乘法结合律和相乘顺序**  
因为矩阵乘法满足结合律(associative laws)，也就是多个矩阵相乘时,相乘的顺序不影响结果,顺序可以用括号来表示,比如$(A_0 * A_1) * A_2 = A_0 * (A_1 * A_2)$,前者表示$A_0, A_1$是第一次矩阵乘法,它们的乘积和$A_2$做第二次矩阵乘法,后者表示$A_1, A_2$先乘.  

**矩阵链乘法问题**
虽然相乘顺序不影响结果,但是影响运算中标量乘法的数量.比如对于$A_0^{1*5} * A_1^{5*10} * A_2^{10*5}$,其中$A_0$有1行5列.   
$(A_0 * A_1) * A_2$需要 $1*5*10 + 1*10*5 = 100$ 标量计算,而$A_0 * (A_1 * A_2)$需要$5*10*5 + 1*5 * 5 = 275$标量运算.所以我们希望找到一个相乘顺序,使得矩阵链乘法需要的标量运算最少.

在这种情况下,可以用动态规划解决.设整个矩阵链最后一次乘法发生在$A_0*A_1*...*A_k$和$A_{k+1}*...*A_l$处,则此时最少的标量运算就是$A_0*A_1*...*A_{k-1}$所需的最少的标量运算加上$A_k*...*A_l$所需的最少的标量运算再加上$D_0*D_k*D_{l+1}$.  
我们从$k=0,...,l-1$中选择最好的k就得到了最优乘法顺序.也就是  
$M[0,l] = \max\limits_{k=0}^{l-1} M[0,k]+M[k+1,l]+D_0*D_{k+1}*D_{l+1}$

我们用M和P两个矩阵来实现动态规划:
1. M表示矩阵乘法所需的最少的标量计算,$M_{i,j}, i <= j$表示$A_i*A_{i+1}*...*A_j$所需的最少的标量积算.  
2. P记录最优乘法顺序时最后一次乘法发生的位置.$P_{i,j} = k, i <= j$表示$A_i*A_{i+1}*...*A_j$最后一次乘法发生在$A_k$左侧.  

所以$M_{0,l}$就是整个矩阵链所需的最少的标量积算,运算顺序可以通过P矩阵倒推.

In [1]:
from MatMul import MatMulOrder

**讲解中例子**

In [2]:
D = [1, 5, 10,5]

In [3]:
M, P = MatMulOrder(D)

There are 3 matrix to multiply


In [4]:
M 

[[0, 50, 100], [0, 0, 250], [0, 0, 0]]

In [5]:
P

[[0, 0, 1], [0, 0, 1], [0, 0, 0]]

对应$A_0^{1*5} * A_1^{5*10} * A_2^{10*5}$的矩阵链乘法,
1. 需要的标量运算值就是M[0,2] = 100
2. 乘法顺序就是最后一次乘法发生在$A_1$右侧,用括号表示就是$(A_0 * A_1) * A_2$

**例子2**

In [6]:
D1 = [5, 15, 10,15,10]

In [7]:
M1, P1 = MatMulOrder(D1)

There are 4 matrix to multiply


In [8]:
M1

[[0, 750, 1500, 2250], [0, 0, 2250, 3000], [0, 0, 0, 1500], [0, 0, 0, 0]]

In [9]:
P1

[[0, 0, 1, 2], [0, 0, 1, 1], [0, 0, 0, 2], [0, 0, 0, 0]]

对应$A_0^{5*15} * A_1^{15*10} * A_2^{10*15} * A_3^{15*10}$的矩阵链乘法,
1. 需要的标量运算值就是M1[0,3] = 2250
2. 乘法顺序就是
    1. 最后一次乘法发生在$A_2$右侧,用括号表示就是$(A_0 * A_1 * A_2) * A_3$
    2. 之前乘法发生在$A_1$右侧,用括号表示就是$( (A_0 * A_1) * A_2) * A_3$

大家可以演算下

此外,我们发现一个违反直觉的事实,就是 $A_0^{5*15} * A_1^{15*10} * A_2^{10*15} * A_3^{15*10}$最少需要2250次标量运算,  
而$ A_1^{15*10} * A_2^{10*15} * A_3^{15*10}$竟然最少需要3000次标量运算！  